# Pothole Detection 2

## Dataset Preparation

In [ ]:
! pip install -q kaggle
# kaggle.json
from google.colab import files
files.upload()

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
!rm -rf dataset

In [4]:
# Download and unzip the Video Dataset
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/5bwfg4v4cd-1.zip -O video_dataset.zip
!unzip -q video_dataset.zip -d /content/video_dataset

--2024-06-05 07:55:33--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/5bwfg4v4cd-1.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 52.92.0.210, 52.92.19.202, 3.5.70.120, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|52.92.0.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1025803823 (978M) [application/zip]
Saving to: ‘video_dataset.zip’

video_dataset.zip   100%[===================>] 978.28M  24.0MB/s    in 40s     

2024-06-05 07:56:14 (24.4 MB/s) - ‘video_dataset.zip’ saved [1025803823/1025803823]



In [5]:
# Download and unzip the Pothole Image Segmentation Dataset from Kaggle
!kaggle datasets download farzadnekouei/pothole-image-segmentation-dataset
!unzip -q pothole-image-segmentation-dataset.zip -d /content/pothole_image_segmentation_dataset
!unzip -q /content/video_dataset/Pothole\ Videos/pothole_video.zip -d /content/video_dataset/Pothole\ Videos

Dataset URL: https://www.kaggle.com/datasets/farzadnekouei/pothole-image-segmentation-dataset
License(s): Apache 2.0
 81% 48.0M/59.3M [00:00<00:00, 122MB/s] 
100% 59.3M/59.3M [00:00<00:00, 82.9MB/s]


In [6]:
# Download and unzip the Pothole Detection Dataset from Kaggle
!kaggle datasets download atulyakumar98/pothole-detection-dataset
!unzip -q pothole-detection-dataset.zip -d /content/pothole_detection_dataset

Dataset URL: https://www.kaggle.com/datasets/atulyakumar98/pothole-detection-dataset
License(s): CC0-1.0
 98% 191M/194M [00:01<00:00, 123MB/s]
100% 194M/194M [00:01<00:00, 114MB/s]


In [7]:
!ls /content/video_dataset/Pothole\ Videos/pothole_video
!ls /content/pothole_image_segmentation_dataset/Pothole_Segmentation_YOLOv8
!ls /content/pothole_detection_dataset

test  train  val
data.yaml  README.dataset.txt  README.roboflow.txt  sample_video.mp4  train  valid
normal	potholes


## Data Preparation

In [9]:
%%capture
!pip install -U ultralytics==8.0.58

In [10]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
from PIL import Image

### Video Dataset

In [11]:
class VideoDataset(Dataset):
    def __init__(self, video_dir, mask_dir):
        self.video_files = sorted(os.listdir(video_dir))
        self.mask_files = sorted(os.listdir(mask_dir))
        self.video_dir = video_dir
        self.mask_dir = mask_dir

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_path = os.path.join(self.video_dir, self.video_files[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])

        video = cv2.VideoCapture(video_path)
        mask = cv2.VideoCapture(mask_path)

        frames = []
        mask_frames = []
        while(video.isOpened()):
            ret, frame = video.read()
            ret_mask, mask_frame = mask.read()
            if ret and ret_mask:
                frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                mask_frames.append(mask_frame)
            else:
                break

        video.release()
        mask.release()

        return np.array(frames), np.array(mask_frames)

def create_video_dataloader(video_dir, mask_dir, batch_size=4):
    dataset = VideoDataset(video_dir, mask_dir)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

train_video_dir = '/content/video_dataset/Pothole Videos/pothole_video/train/rgb'
train_mask_dir = '/content/video_dataset/Pothole Videos/pothole_video/train/mask'

val_video_dir = '/content/video_dataset/Pothole Videos/pothole_video/val/rgb'
val_mask_dir = '/content/video_dataset/Pothole Videos/pothole_video/val/mask'

test_video_dir = '/content/video_dataset/Pothole Videos/pothole_video/test/rgb'
test_mask_dir = '/content/video_dataset/Pothole Videos/pothole_video/test/mask'

train_dataloader = create_video_dataloader(train_video_dir, train_mask_dir)
val_dataloader = create_video_dataloader(val_video_dir, val_mask_dir)
test_dataloader = create_video_dataloader(test_video_dir, test_mask_dir)

### Pothole Image Segmentation Dataset

In [12]:
segmentation_dataset_path = '/content/pothole_image_segmentation_dataset/Pothole_Segmentation_YOLOv8'

model = YOLO('yolov8n-seg.pt')
yaml_path = os.path.join(segmentation_dataset_path, 'data.yaml')

model.train(data=yaml_path, epochs=10, imgsz=640, seed=42)

100%|██████████| 6.73M/6.73M [00:00<00:00, 118MB/s]
New https://pypi.org/project/ultralytics/8.2.28 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.58 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/pothole_image_segmentation_dataset/Pothole_Segmentation_YOLOv8/data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=42, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_c

### Pothole Detection Dataset

In [13]:
class PotholeDetectionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = []
        self.labels = []

        for label in ['normal', 'potholes']:
            folder = os.path.join(root_dir, label)
            for img_file in os.listdir(folder):
                self.image_files.append(os.path.join(folder, img_file))
                self.labels.append(0 if label == 'normal' else 1)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

detection_dataset_path = '/content/pothole_detection_dataset'
detection_dataset = PotholeDetectionDataset(detection_dataset_path, transform=transform)
detection_dataloader = DataLoader(detection_dataset, batch_size=16, shuffle=True)

## Training

In [14]:
def combined_training(model, train_dataloader, val_dataloader, detection_dataloader, epochs=10):
    best_loss = float('inf')
    best_model_path = '/content/best_pothole_detection_model.pt'

    for epoch in range(epochs):
        model.train()

        # Train on video data
        for video_batch in train_dataloader:
            videos, masks = video_batch
            model.train_step(videos, masks)

        # Train on image data
        for image_batch in val_dataloader:
            images, masks = image_batch
            model.train_step(images, masks)

        # Train on detection data
        for detection_batch in detection_dataloader:
            images, labels = detection_batch
            model.train_step(images, labels)

        # Validate the model
        val_loss = 0
        model.eval()
        with torch.no_grad():
            # Validate on video data
            for val_video_batch in val_dataloader:
                val_videos, val_masks = val_video_batch
                val_loss += model.val_step(val_videos, val_masks)

            # Validate on image data
            for val_image_batch in val_dataloader:
                val_images, val_masks = val_image_batch
                val_loss += model.val_step(val_images, val_masks)

            # Validate on detection data
            for val_detection_batch in detection_dataloader:
                val_images, val_labels = val_detection_batch
                val_loss += model.val_step(val_images, val_labels)

        # Save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            model.save(best_model_path)

# Perform combined training
combined_training(model, train_dataloader, val_dataloader, detection_dataloader)

New https://pypi.org/project/ultralytics/8.2.28 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.58 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/pothole_image_segmentation_dataset/Pothole_Segmentation_YOLOv8/data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=42, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize

AttributeError: 'YOLO' object has no attribute 'train_step'. See valid attributes below.

    YOLO (You Only Look Once) object detection model.

    Args:
        model (str, Path): Path to the model file to load or create.

    Attributes:
        predictor (Any): The predictor object.
        model (Any): The model object.
        trainer (Any): The trainer object.
        task (str): The type of model task.
        ckpt (Any): The checkpoint object if the model loaded from *.pt file.
        cfg (str): The model configuration if loaded from *.yaml file.
        ckpt_path (str): The checkpoint file path.
        overrides (dict): Overrides for the trainer object.
        metrics (Any): The data for metrics.

    Methods:
        __call__(source=None, stream=False, **kwargs):
            Alias for the predict method.
        _new(cfg:str, verbose:bool=True) -> None:
            Initializes a new model and infers the task type from the model definitions.
        _load(weights:str, task

In [ ]:
def calculate_iou(pred, target, threshold=0.5):
    pred = (pred > threshold).astype(np.uint8)
    intersection = np.logical_and(pred, target)
    union = np.logical_or(pred, target)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def calculate_pixel_accuracy(pred, target, threshold=0.5):
    pred = (pred > threshold).astype(np.uint8)
    correct = np.sum(pred == target)
    total = target.size
    accuracy = correct / total
    return accuracy

In [ ]:
class PotholeDetectionEvaluator:
    def __init__(self, model, test_dataloader):
        self.model = model
        self.test_dataloader = test_dataloader

    def evaluate(self):
        total_iou = 0
        total_accuracy = 0
        total_videos = 0

        self.model.eval()
        with torch.no_grad():
            for test_batch in self.test_dataloader:
                test_videos, test_masks = test_batch
                for video, mask in zip(test_videos, test_masks):
                    predictions = self.model.predict(video)
                    iou_scores = []
                    accuracies = []

                    for pred_frame, mask_frame in zip(predictions, mask):
                        iou = calculate_iou(pred_frame, mask_frame)
                        accuracy = calculate_pixel_accuracy(pred_frame, mask_frame)
                        iou_scores.append(iou)
                        accuracies.append(accuracy)

                    avg_iou = np.mean(iou_scores)
                    avg_accuracy = np.mean(accuracies)

                    total_iou += avg_iou
                    total_accuracy += avg_accuracy
                    total_videos += 1

        avg_iou = total_iou / total_videos
        avg_accuracy = total_accuracy / total_videos
        print(f"Average IoU: {avg_iou:.4f}")
        print(f"Average Pixel Accuracy: {avg_accuracy:.4f}")

evaluator = PotholeDetectionEvaluator(best_model, test_dataloader)

evaluator.evaluate()